# Ranking and normalising ESG Scores And creating a Green minus Brown portfolio

# Imports

In [1]:
import pandas as pd 
import numpy as np
import os
from pathlib import Path
from collections import Counter
from datetime import datetime, timedelta

dataPath = str(Path(os.getcwd()).parent.absolute())+"/data"

# Reading the data

In [2]:
stocksData = pd.read_csv(dataPath+'/raw/stocksData.csv', index_col=0) 

In [3]:
stocksNormalizedData = stocksData.copy()

In [4]:
#deleteting the value and changing the type from int to float for the esg ranking
for column in stocksData:
        if column[-22:] == "|ESG_RANKING_ROBECOSAM":
            stocksNormalizedData[column]=0.0

# Ranking and normalising Robecosam Environmental Ranking
Robecosam Environnental Ranking scores companies from 100 (very green) to 0 (very brown)

The following code will rank and normalize those scores between 0 and 1 so that 10% of companies have a score above 0.9, 0 being the best (green) and 1 the worst (brown) (not enterely true due to companies having the same ranking)

In [5]:
#Very slow, not optimized
for index, row in stocksData.iterrows():
    dailyESGRankings=[]
    for column in stocksData:
        if column[-22:] == "|ESG_RANKING_ROBECOSAM":
            dailyESGRankings+=[stocksData[column][index]]
    dailyESGRankings.sort(reverse=True)

    #important step: if the last stocks are ex aequo put them in last place with a score of zero, so that we always have stocks for the making of the brown portfolio
    # if we don't do this those 2 stocks will not have a score of zero, they will have a score of (nb of stock-1)/(nb of stocks)
    # this could cause issue if our starting portfolio doesnt contains many stocks
    # this is not the case here, but it might when will create the green minus brown portfolios per industry later
    
    worstScoreRank=-1
    if len(dailyESGRankings)-1!= dailyESGRankings.index(dailyESGRankings[-1]):
        worstScoreRank=dailyESGRankings.index(dailyESGRankings[-1])
    
    for column in stocksData:
        if column[-22:] == "|ESG_RANKING_ROBECOSAM":
            
            if dailyESGRankings.index(stocksData[column][index]) == worstScoreRank:
                stocksNormalizedData[column][index]=1
            else:
                stocksNormalizedData[column][index] = dailyESGRankings.index(stocksData[column][index])/len(dailyESGRankings) #ISSUE IF LOT OF COMAPNIES HAVE THE SAME SCORE ?!!!!?
    

/tmp/ipykernel_167079/1326309656.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksNormalizedData[column][index] = dailyESGRankings.index(stocksData[column][index])/len(dailyESGRankings) #ISSUE IF LOT OF COMAPNIES HAVE THE SAME SCORE ?!!!!?
/tmp/ipykernel_167079/1326309656.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksNormalizedData[column][index]=1


# Ranking and normalising Bloomberg Environmental Ranking
Bloomberg Environnental Score 
The following code will rank and normalize those scores between 0 and 1 so that 10% of companies have a score above 0.9, 0 being the best (green) and 1 the worst (brown) (not enterely true due to companies having the same ranking)

In [6]:
#Not optimized very slow

for index, row in stocksData.iterrows():
    dailyESGRankings=[]

    for column in stocksData:
        if column[-20:] == "|ESG_SCORE_BLOOMBERG":
            dailyESGRankings+=[stocksData[column][index]]
    dailyESGRankings.sort(reverse=True)
    
    
    #important step: if the last stocks are ex aequo put them in last place with a score of zero, so that we always have stocks for the making of the brown portfolio
    # if we don't do this those 2 stocks will not have a score of zero, they will have a score of (nb of stock-1)/(nb of stocks)
    # this could cause issue if our starting portfolio doesnt contains many stocks
    # this is not the case here, but it might when will create the green minus brown portfolios per industry later
    
    worstScoreRank=-1
    if len(dailyESGRankings)-1!= dailyESGRankings.index(dailyESGRankings[-1]):
        worstScoreRank=dailyESGRankings.index(dailyESGRankings[-1])
    
    for column in stocksData:
        if column[-20:] == "|ESG_SCORE_BLOOMBERG":
            
            if dailyESGRankings.index(stocksData[column][index]) == worstScoreRank:
                stocksNormalizedData[column][index]=1
            else:
                stocksNormalizedData[column][index] = dailyESGRankings.index(stocksData[column][index])/len(dailyESGRankings) #ISSUE IF LOT OF COMAPNIES HAVE THE SAME SCORE ?!!!!?

    

/tmp/ipykernel_167079/2627010068.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksNormalizedData[column][index] = dailyESGRankings.index(stocksData[column][index])/len(dailyESGRankings) #ISSUE IF LOT OF COMAPNIES HAVE THE SAME SCORE ?!!!!?
/tmp/ipykernel_167079/2627010068.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksNormalizedData[column][index]=1


# Making the green minus brown portfolio
Long the best 30% stocks and short the worst 30% stocks
!!!!!  due to the fact that there might be ex aequo it might be a bit more than 30% for the green stocks, and a bit less than 30% for for the brown ones
example:
if we have a portfolio of 3 stocks A, B, C

### if A, B, C have scores of 0, 0.5, 1 respectively: (meaning C is the brownest A the greenest and B is in the middle)
The green portfolio will be constructed of all stocks with a score inferior to 0.3  -> stock A

The brown portfolio will be constructed of all stocks with a score superior to 0.7 -> stock C

### if A, B, C have scores of 0, 0, 1 respectively: (meaning A and B are the greenest and C the brownest )
The green portfolio will be constructed of all stocks with a score inferior to 0.3 -> stock A and B <i>(with an equal weight. every stock in green or brown portfolio will have the same weight, there are other possible methodologies such as having a bigger weigh for stock with lower score close to 0 and smaller weight for stocks with a score closer to 0.3, this could be further studied in futur work)</i>

The brown portfolio will be constructed of all stocks with a score superior to 0.7 -> stock C

Method:
for each day, we count the nb of stocks with a score <0.3 (for day D-1 !!!) , sum the daily returns of those stock (for day D !!!) and create the daily returns of the green portfolio: summedReturns/nbOfStocks

In [7]:
marketGreenPortfolios = pd.DataFrame(index=stocksData.index, columns=["MKT_GREEN_BL|DAILY_RETURNS","MKT_BROWN_BL|DAILY_RETURNS","MKT_GMB_BL|DAILY_RETURNS","MKT_GREEN_RO|DAILY_RETURNS","MKT_BROWN_RO|DAILY_RETURNS","MKT_GMB_RO|DAILY_RETURNS"])

lastDay=stocksData.index[0]
for index, row in stocksData[stocksData.index > stocksData.index[0]].iterrows(): #skip the fisrt day
    countBloombergGreenStocks, countBloombergBrownStocks, countRobecosamGreenStocks, countRobecosamBrownStocks = 0, 0, 0 ,0
    returnsBloombergGreenStocks, returnsBloombergBrownStocks, returnsRobecosamGreenStocks, returnsRobecosamBrownStocks = 0, 0, 0, 0
#     print(index)
    for column in stocksNormalizedData:
        if column[-20:] == "|ESG_SCORE_BLOOMBERG":
            if stocksNormalizedData[column][lastDay]<0.3:
                countBloombergGreenStocks+=1
#                 print(column,stocksNormalizedData[column[:-20]+"|DAILY_RETURNS"][index])
                
                returnsBloombergGreenStocks+=stocksNormalizedData[column[:-20]+"|DAILY_RETURNS"][index]
            elif stocksNormalizedData[column][lastDay]>0.7:
                countBloombergBrownStocks+=1
                returnsBloombergBrownStocks+=stocksNormalizedData[column[:-20]+"|DAILY_RETURNS"][index]
        elif column[-22:] == "|ESG_RANKING_ROBECOSAM":
            if stocksNormalizedData[column][lastDay]<0.3:
                countRobecosamGreenStocks+=1
                returnsRobecosamGreenStocks+=stocksNormalizedData[column[:-22]+"|DAILY_RETURNS"][index]
            elif stocksNormalizedData[column][lastDay]>0.7:
                countRobecosamBrownStocks+=1
                returnsRobecosamBrownStocks+=stocksNormalizedData[column[:-22]+"|DAILY_RETURNS"][index]
                
#         print(countBloombergGreenStocks, countBloombergBrownStocks, countRobecosamGreenStocks, countRobecosamBrownStocks)
        
    marketGreenPortfolios["MKT_GREEN_BL|DAILY_RETURNS"][index]=returnsBloombergGreenStocks/countBloombergGreenStocks
    marketGreenPortfolios["MKT_BROWN_BL|DAILY_RETURNS"][index]=returnsBloombergBrownStocks/countBloombergBrownStocks
    marketGreenPortfolios["MKT_GREEN_RO|DAILY_RETURNS"][index]=returnsRobecosamGreenStocks/countRobecosamGreenStocks
    marketGreenPortfolios["MKT_BROWN_RO|DAILY_RETURNS"][index]=returnsRobecosamBrownStocks/countRobecosamBrownStocks
    
        
        
                
        
    

In [10]:
marketGreenPortfolios["MKT_GMB_BL|DAILY_RETURNS"]=marketGreenPortfolios["MKT_GREEN_BL|DAILY_RETURNS"]- marketGreenPortfolios["MKT_BROWN_BL|DAILY_RETURNS"]
marketGreenPortfolios["MKT_GMB_RO|DAILY_RETURNS"]=marketGreenPortfolios["MKT_GREEN_RO|DAILY_RETURNS"]- marketGreenPortfolios["MKT_BROWN_RO|DAILY_RETURNS"]

In [11]:
marketGreenPortfolios["MKT_BROWN|DAILY_RETURNS"]=marketGreenPortfolios["MKT_BROWN_BL|DAILY_RETURNS"]/2+marketGreenPortfolios["MKT_BROWN_RO|DAILY_RETURNS"]/2
marketGreenPortfolios["MKT_GREEN|DAILY_RETURNS"]=marketGreenPortfolios["MKT_GREEN_BL|DAILY_RETURNS"]/2+marketGreenPortfolios["MKT_GREEN_RO|DAILY_RETURNS"]/2
marketGreenPortfolios["MKT_GMB|DAILY_RETURNS"]=marketGreenPortfolios["MKT_GMB_BL|DAILY_RETURNS"]/2+marketGreenPortfolios["MKT_GMB_RO|DAILY_RETURNS"]/2


In [12]:
marketGreenPortfolios.to_csv(dataPath+'/interim/marketGreenPortfolios.csv')